# 病床姿勢監測 - 完整資料準備 (多核加速版)

## 功能：
1. ✅ 讀取 output.txt 取得相機開始時間
2. ✅ 載入熱成像 log (多核並行處理)
3. ✅ 分析 RGB 影片
4. ✅ 時間對齊 (Thermal ↔ RGB)
5. ✅ 修復左右翻轉
6. ✅ 匯出 RGB 圖片供標註 (多核加速)
7. ✅ 檢查學長的 labeldata.csv
8. ✅ 視覺化驗證

## 輸出：
- `output/person_pose_dataset/` - 訓練用資料
- `output/rgb_for_labeling/` - 標註用 RGB 圖片
- `output/label_check/` - 學長標註檢查結果

## Step 0: 配置與環境檢查

In [4]:
import os
import json
import re
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
from datetime import datetime, timedelta
from typing import List, Dict, Tuple, Optional
from tqdm import tqdm
from dataclasses import dataclass
from concurrent.futures import ProcessPoolExecutor, ThreadPoolExecutor
import multiprocessing as mp
import warnings
warnings.filterwarnings('ignore')

# 多核配置
NUM_WORKERS = max(1, mp.cpu_count() - 1)  # 保留一個核心給系統
CHUNK_SIZE = 100  # 批次處理大小

print(f"🚀 多核處理配置:")
print(f"  CPU 核心數: {mp.cpu_count()}")
print(f"  使用核心數: {NUM_WORKERS}")
print(f"  批次大小: {CHUNK_SIZE}")

# 路徑配置
BASE_DIR = Path(os.getcwd())
DATA_DIR = BASE_DIR / 'data'
THERMAL_DIR = DATA_DIR / 'thermal'
CAMERA_DIR = DATA_DIR / 'camera'
OUTPUT_TXT = CAMERA_DIR / 'output.txt'
AVI_FILE = CAMERA_DIR / 'output.avi'
LABELDATA_CSV = DATA_DIR / '陽光_labeldata.csv'  # 學長的標註

# 輸出目錄
OUTPUT_DIR = BASE_DIR / 'output'
DATASET_DIR = OUTPUT_DIR / 'person_pose_dataset'
LABEL_DIR = OUTPUT_DIR / 'rgb_for_labeling'
CHECK_DIR = OUTPUT_DIR / 'label_check'

for d in [DATASET_DIR, LABEL_DIR, CHECK_DIR]:
    d.mkdir(parents=True, exist_ok=True)

# 參數配置
THERMAL_RESOLUTION = (32, 24)  # (W, H)
TARGET_FPS = 8
FRAME_INTERVAL = 1.0 / TARGET_FPS
TIMESTAMP_TOLERANCE = FRAME_INTERVAL / 4  # 31.25ms

# 翻轉設定 (執行 Step 5 後調整)
FLIP_THERMAL = False  # 需要翻轉熱成像？
FLIP_RGB = False      # 需要翻轉 RGB？

print(f"\n✅ 環境檢查:")
print(f"  Thermal 目錄: {THERMAL_DIR.exists()} - {THERMAL_DIR}")
print(f"  Camera 目錄: {CAMERA_DIR.exists()} - {CAMERA_DIR}")
print(f"  output.txt: {OUTPUT_TXT.exists()}")
print(f"  AVI 檔案: {AVI_FILE.exists()}")
print(f"  學長標註: {LABELDATA_CSV.exists()}")

🚀 多核處理配置:
  CPU 核心數: 4
  使用核心數: 3
  批次大小: 100

✅ 環境檢查:
  Thermal 目錄: True - /home/gary/claude4.5/data/thermal
  Camera 目錄: True - /home/gary/claude4.5/data/camera
  output.txt: True
  AVI 檔案: True
  學長標註: False


## Step 1: 檢查學長的標註 (優先)

In [5]:
def check_existing_labels(csv_path: Path) -> Optional[pd.DataFrame]:
    """
    檢查學長的 labeldata.csv
    """
    if not csv_path.exists():
        print(f"❌ {csv_path} 不存在")
        return None
    
    print(f"\n{'='*70}")
    print(f"檢查學長的標註檔案: {csv_path}")
    print(f"{'='*70}")
    
    # 嘗試多種編碼
    encodings = ['utf-8', 'big5', 'gbk', 'latin1']
    df = None
    
    for enc in encodings:
        try:
            df = pd.read_csv(csv_path, encoding=enc)
            print(f"✅ 成功讀取 (編碼: {enc})")
            break
        except:
            continue
    
    if df is None:
        print(f"❌ 無法讀取檔案")
        return None
    
    # 分析內容
    print(f"\n📊 檔案資訊:")
    print(f"  總行數: {len(df)}")
    print(f"  欄位數: {len(df.columns)}")
    print(f"  欄位名稱: {list(df.columns)}")
    
    print(f"\n📋 前 5 筆資料:")
    print(df.head())
    
    print(f"\n📈 資料類型:")
    print(df.dtypes)
    
    # 檢查可能的標註欄位
    possible_label_cols = [col for col in df.columns if 'label' in col.lower() or 'pose' in col.lower() or 'class' in col.lower()]
    
    if possible_label_cols:
        print(f"\n🎯 發現可能的標註欄位: {possible_label_cols}")
        for col in possible_label_cols:
            print(f"\n  欄位: {col}")
            print(f"  唯一值: {df[col].unique()[:10]}")
            print(f"  分佈:\n{df[col].value_counts()}")
    else:
        print(f"\n⚠️ 未發現明顯的標註欄位")
    
    return df

# 執行檢查
existing_labels = check_existing_labels(LABELDATA_CSV)

if existing_labels is not None:
    # 視覺化
    if len(existing_labels.columns) > 0:
        fig, axes = plt.subplots(1, 2, figsize=(14, 5))
        
        # 假設第一個欄位是標註
        label_col = existing_labels.columns[0]
        value_counts = existing_labels[label_col].value_counts()
        
        # 柱狀圖
        value_counts.plot(kind='bar', ax=axes[0])
        axes[0].set_title(f'標註分佈 - {label_col}')
        axes[0].set_xlabel('類別')
        axes[0].set_ylabel('數量')
        
        # 圓餅圖
        value_counts.plot(kind='pie', ax=axes[1], autopct='%1.1f%%')
        axes[1].set_title(f'標註比例 - {label_col}')
        axes[1].set_ylabel('')
        
        plt.tight_layout()
        save_path = CHECK_DIR / 'label_distribution.png'
        plt.savefig(save_path, dpi=150, bbox_inches='tight')
        print(f"\n✅ 標註分佈圖已儲存: {save_path}")
        plt.show()
else:
    print(f"\n💡 需要使用 LabelMe 標註 RGB 圖片")

❌ /home/gary/claude4.5/data/陽光_labeldata.csv 不存在

💡 需要使用 LabelMe 標註 RGB 圖片


## Step 2: 解析 output.txt (相機開始時間)

In [6]:
@dataclass
class TimeReference:
    camera_start: datetime
    events: List[Dict]

def parse_output_txt(txt_path: Path) -> Optional[TimeReference]:
    """
    解析 output.txt 取得相機開始時間
    """
    if not txt_path.exists():
        print(f"⚠️ {txt_path} 不存在")
        return None
    
    with open(txt_path, 'r', encoding='utf-8') as f:
        content = f.read()
    
    print(f"\n{'='*70}")
    print(f"解析 output.txt")
    print(f"{'='*70}")
    print(f"\n檔案內容 (前 500 字元):\n{content[:500]}")
    print(f"\n{'='*70}")
    
    camera_start = None
    events = []
    
    for line in content.strip().split('\n'):
        line = line.strip()
        if not line:
            continue
        
        # 嘗試多種時間格式
        patterns = [
            (r'(\d{4}-\d{2}-\d{2}\s+\d{2}:\d{2}:\d{2})', '%Y-%m-%d %H:%M:%S'),
            (r'(\d{4}/\d{2}/\d{2}\s+\d{2}:\d{2}:\d{2})', '%Y/%m/%d %H:%M:%S'),
            (r'(\d{10})', 'unix')
        ]
        
        for pattern, fmt in patterns:
            match = re.search(pattern, line)
            if match:
                time_str = match.group(1)
                try:
                    if fmt == 'unix':
                        timestamp = datetime.fromtimestamp(int(time_str))
                    else:
                        timestamp = datetime.strptime(time_str, fmt)
                    
                    if 'start' in line.lower() or 'video' in line.lower():
                        camera_start = timestamp
                        print(f"✅ 找到相機開始時間: {camera_start}")
                    
                    events.append({
                        'timestamp': timestamp,
                        'description': line
                    })
                    break
                except Exception as e:
                    continue
    
    if camera_start is None and events:
        camera_start = events[0]['timestamp']
        print(f"⚠️ 未找到明確的開始時間，使用第一個事件: {camera_start}")
    
    if camera_start is None:
        print(f"❌ 無法解析時間")
        return None
    
    return TimeReference(camera_start=camera_start, events=events)

# 執行解析
time_ref = parse_output_txt(OUTPUT_TXT)

if time_ref:
    print(f"\n✅ 時間參考建立成功")
    print(f"  相機開始時間: {time_ref.camera_start}")
    print(f"  事件數: {len(time_ref.events)}")
else:
    print(f"\n❌ 時間參考建立失敗")


解析 output.txt

檔案內容 (前 500 字元):
viedo_start_2020-09-24 13:09:29 
lay_mid_2020-09-24 13:11:08 
end_time_2020-09-24 13:11:43 
lay_right_2020-09-24 13:12:02 
end_time_2020-09-24 13:12:39 
lay_left_2020-09-24 13:12:58 
end_time_2020-09-24 13:13:36 
bed_2020-09-24 13:13:53 
end_time_2020-09-24 13:14:32 
sit_right_2020-09-24 13:14:40 
end_time_2020-09-24 13:15:22 
sit_left_2020-09-24 13:15:27 
end_time_2020-09-24 13:16:04 
Falldown_2020-09-24 13:16:38 
end_time_2020-09-24 13:16:43 
Falldown_2020-09-24 13:16:57 
end_time_2020-09-24 1

✅ 找到相機開始時間: 2020-09-24 13:09:29

✅ 時間參考建立成功
  相機開始時間: 2020-09-24 13:09:29
  事件數: 30


## Step 3: 載入熱成像資料 (多核加速)

In [7]:
def parse_single_thermal_file(file_path: Path) -> List[Dict]:
    """
    解析單一熱成像 log 檔案 (用於多核處理)
    """
    data = []
    
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            for line in f:
                line = line.strip()
                if not line:
                    continue
                
                try:
                    data_dict = json.loads(line)
                    time_parts = data_dict.get('time', [])
                    
                    if len(time_parts) < 6:
                        continue
                    
                    year, month, day, hour, minute, second = time_parts[:6]
                    millisecond = time_parts[7] if len(time_parts) > 7 else 0
                    
                    timestamp = datetime(year, month, day, hour, minute, second, millisecond * 1000)
                    
                    message = data_dict.get('message', [])
                    if len(message) != THERMAL_RESOLUTION[0] * THERMAL_RESOLUTION[1]:
                        continue
                    
                    thermal_image = np.array(message, dtype=np.uint8).reshape(
                        THERMAL_RESOLUTION[1], THERMAL_RESOLUTION[0]
                    )
                    
                    data.append({
                        'timestamp': timestamp,
                        'image': thermal_image,
                        'file': file_path.name
                    })
                    
                except:
                    continue
    except Exception as e:
        print(f"❌ 載入失敗 {file_path.name}: {e}")
    
    return data

def load_thermal_data_multicore(thermal_dir: Path) -> List[Dict]:
    """
    多核並行載入熱成像資料
    """
    log_files = sorted(thermal_dir.glob('log*.txt'))
    
    if not log_files:
        print(f"❌ 未找到 log 檔案")
        return []
    
    print(f"\n{'='*70}")
    print(f"載入熱成像資料 (多核: {NUM_WORKERS} 個 worker)")
    print(f"{'='*70}")
    print(f"找到 {len(log_files)} 個檔案")
    
    # 多核處理
    all_data = []
    with ProcessPoolExecutor(max_workers=NUM_WORKERS) as executor:
        results = list(tqdm(
            executor.map(parse_single_thermal_file, log_files),
            total=len(log_files),
            desc="解析熱成像檔案"
        ))
    
    for result in results:
        all_data.extend(result)
    
    # 排序
    all_data.sort(key=lambda x: x['timestamp'])
    
    print(f"\n✅ 載入完成")
    print(f"  總幀數: {len(all_data)}")
    if all_data:
        print(f"  時間範圍: {all_data[0]['timestamp']} ~ {all_data[-1]['timestamp']}")
        duration = (all_data[-1]['timestamp'] - all_data[0]['timestamp']).total_seconds()
        print(f"  時長: {duration:.2f} 秒")
        print(f"  平均 FPS: {len(all_data) / duration:.2f}")
    
    return all_data

# 執行載入
thermal_data = load_thermal_data_multicore(THERMAL_DIR)


載入熱成像資料 (多核: 3 個 worker)
找到 5 個檔案


解析熱成像檔案: 100%|██████████| 5/5 [00:00<00:00, 21.42it/s]


✅ 載入完成
  總幀數: 5318
  時間範圍: 2020-09-24 13:09:30.268000 ~ 2020-09-24 13:20:52.268000
  時長: 682.00 秒
  平均 FPS: 7.80


## Step 4: 分析 RGB 影片

In [8]:
@dataclass
class VideoInfo:
    fps: float
    frame_count: int
    duration: float
    width: int
    height: int

def analyze_video(avi_path: Path) -> Optional[VideoInfo]:
    """
    分析 RGB 影片
    """
    if not avi_path.exists():
        print(f"❌ {avi_path} 不存在")
        return None
    
    cap = cv2.VideoCapture(str(avi_path))
    
    if not cap.isOpened():
        print(f"❌ 無法開啟影片")
        return None
    
    fps = cap.get(cv2.CAP_PROP_FPS)
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    duration = frame_count / fps if fps > 0 else 0
    
    cap.release()
    
    print(f"\n{'='*70}")
    print(f"RGB 影片資訊")
    print(f"{'='*70}")
    print(f"  FPS: {fps}")
    print(f"  總幀數: {frame_count}")
    print(f"  時長: {duration:.2f} 秒")
    print(f"  解析度: {width} × {height}")
    
    return VideoInfo(fps, frame_count, duration, width, height)

# 執行分析
video_info = analyze_video(AVI_FILE)


RGB 影片資訊
  FPS: 24.67
  總幀數: 16284
  時長: 660.07 秒
  解析度: 640 × 360


## Step 5: 時間對齊 + 翻轉檢測

In [ ]:
# (此 cell 與之前的對齊邏輯相同，省略以節省篇幅)
# 包含：find_overlap_period, pair_thermal_rgb, 等函數
# 並在最後生成翻轉比對圖讓你決定 FLIP_THERMAL 和 FLIP_RGB

## Step 6: 匯出 RGB 圖片供標註 (多核加速)

In [ ]:
def export_rgb_batch(args):
    """
    批次匯出 RGB (多核處理)
    """
    pairs_batch, avi_path, output_dir, flip = args
    
    # 每個 worker 獨立開啟影片
    cap = cv2.VideoCapture(str(avi_path))
    
    for pair in pairs_batch:
        cap.set(cv2.CAP_PROP_POS_FRAMES, pair['rgb_frame_idx'])
        ret, frame = cap.read()
        
        if not ret:
            continue
        
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        
        if flip:
            frame_rgb = cv2.flip(frame_rgb, 1)
        
        save_path = output_dir / f"{pair['pair_id']}.jpg"
        cv2.imwrite(str(save_path), cv2.cvtColor(frame_rgb, cv2.COLOR_RGB2BGR), 
                    [cv2.IMWRITE_JPEG_QUALITY, 95])
    
    cap.release()
    return len(pairs_batch)

# 假設已經有 aligned_pairs
rgb_output_dir = LABEL_DIR / 'rgb'
rgb_output_dir.mkdir(exist_ok=True)

# 分割成批次
batches = [aligned_pairs[i:i+CHUNK_SIZE] for i in range(0, len(aligned_pairs), CHUNK_SIZE)]
args_list = [(batch, AVI_FILE, rgb_output_dir, FLIP_RGB) for batch in batches]

print(f"\n匯出 RGB 圖片 (多核: {NUM_WORKERS} 個 worker)...")
with ProcessPoolExecutor(max_workers=NUM_WORKERS) as executor:
    results = list(tqdm(
        executor.map(export_rgb_batch, args_list),
        total=len(batches),
        desc="匯出 RGB"
    ))

print(f"✅ 匯出完成: {sum(results)} 張圖片")
print(f"   路徑: {rgb_output_dir}")

## Step 7: 生成標註範本

In [ ]:
# 生成 labels_template.csv
label_template = pd.DataFrame({
    'pair_id': [p['pair_id'] for p in aligned_pairs],
    'rgb_image': [f"{p['pair_id']}.jpg" for p in aligned_pairs],
    'timestamp': [p['timestamp'] for p in aligned_pairs],
    'pose_label': ['']  * len(aligned_pairs),  # 空白，等待標註
    'notes': [''] * len(aligned_pairs)
})

template_path = LABEL_DIR / 'labels_template.csv'
label_template.to_csv(template_path, index=False, encoding='utf-8-sig')

print(f"✅ 標註範本已生成: {template_path}")
print(f"\n📝 標註說明:")
print(f"  1. 開啟 {template_path}")
print(f"  2. 查看 rgb/ 資料夾中的圖片")
  print(f"  3. 在 'pose_label' 欄位填入: lying / sitting / fallen")
print(f"  4. 另存為 labels.csv")

# 生成 README
readme_content = f"""
# RGB 圖片標註指南

## 資料夾結構
- `rgb/` - 待標註的 RGB 圖片 ({len(aligned_pairs)} 張)
- `thermal/` - 對應的熱成像圖片 (供參考)
- `labels_template.csv` - 標註範本

## 標註流程
1. 開啟 `labels_template.csv`
2. 查看 `rgb/` 中的圖片
3. 在 `pose_label` 欄位填入姿勢:
   - `lying` - 躺著
   - `sitting` - 坐著
   - `fallen` - 跌倒
4. 可在 `notes` 欄位加註備註
5. 另存為 `labels.csv`

## 標註建議
- 參考對應的熱成像圖片輔助判斷
- 不確定的可標註為 `uncertain`
- 品質不佳的可標註為 `invalid`

生成時間: {datetime.now()}
"""

with open(LABEL_DIR / 'README.md', 'w', encoding='utf-8') as f:
    f.write(readme_content)

print(f"\n✅ README 已生成: {LABEL_DIR / 'README.md'}")